## API Funkcjonalne

### Tworzenie złożonych modeli w Keras

Modele w stylu `Sequential` są proste i intuicyjne, ale nie pozwalają na tworzenie bardziej złożonych struktur. W takich przypadkach można skorzystać z API funkcjonalnego. Pozwala ono na tworzenie dowolnych struktur modeli, w tym modeli z wieloma wejściami i wyjściami, modeli z warstwami współdzielonymi, itp.

In [2]:
# Uciszenie ostrzeżeń z TensorFlow
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from keras import layers, models

# Definiowanie modelu z użyciem funkcjonalnego API
# Wejście
inputs = layers.Input(shape=(32, 32, 3))

# Pierwsza gałąź
x = layers.Conv2D(32, 3, activation="relu")(inputs)
x = layers.MaxPooling2D(2)(x)

# Druga gałąź
y = layers.Conv2D(64, 3, activation="relu")(inputs)
y = layers.MaxPooling2D(2)(y)

# Połączenie gałęzi
z = layers.Concatenate()([x, y])
z = layers.Conv2D(64, 3, activation="relu")(z)

# Warstwa globalnego uśredniania
# (bs, x, y, c) -> (bs, c)
z = layers.GlobalAveragePooling2D()(z)

# Warstwa klasyfikacji
outputs = layers.Dense(10, activation="softmax")(z)

# Budowanie modelu
# (wejście, wyjście)
model = models.Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 30, 30,    │        896 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 30, 30,    │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 15, 15,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 15, 15,    │          0 │ conv2d_4[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 15, 15,    │          0 │ max_pooling2d_2[… │
│ (Concatenate)       │ 96)               │            │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 13, 13,    │     55,360 │ concatenate_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ conv2d_5[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 10)        │        650 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 58,698 (229.29 KB)

 Trainable params: 58,698 (229.29 KB)

 Non-trainable params: 0 (0.00 B)

### Architektura U-Net za pomocą API funkcjonalnego

In [9]:
# Uciszenie ostrzeżeń z TensorFlow
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
from typing import Sequence
from keras import layers, models

# Budowanie architektury U-Net
inputs = x = layers.Input(shape=(32, 32, 3))

koder: Sequence[layers.Layer] = []
for _ in range(3):
    x = layers.Conv2D(32, 3, activation="relu", padding="same")(x)
    x = layers.MaxPooling2D(2)(x)
    koder.append(x)

x = layers.Conv2D(64, 3, activation="relu", padding="same")(koder[-1])

for i in reversed(range(3)):
    x = layers.Concatenate()([x, koder[i]])
    x = layers.Conv2D(32, 3, activation="relu", padding="same")(x)
    x = layers.UpSampling2D(2)(x)

x = layers.Conv2D(32, 3, activation="relu", padding="same")(x)
outputs = layers.Conv2D(1, 3, activation="sigmoid", padding="same")(x)

model = models.Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_40 (Conv2D)  │ (None, 32, 32,    │        896 │ input_layer_8[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_22    │ (None, 16, 16,    │          0 │ conv2d_40[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_41 (Conv2D)  │ (None, 16, 16,    │      9,248 │ max_pooling2d_22… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_23    │ (None, 8, 8, 32)  │          0 │ conv2d_41[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_42 (Conv2D)  │ (None, 8, 8, 32)  │      9,248 │ max_pooling2d_23… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_24    │ (None, 4, 4, 32)  │          0 │ conv2d_42[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_43 (Conv2D)  │ (None, 4, 4, 64)  │     18,496 │ max_pooling2d_24… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_10      │ (None, 4, 4, 96)  │          0 │ conv2d_43[0][0],  │
│ (Concatenate)       │                   │            │ max_pooling2d_24… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_44 (Conv2D)  │ (None, 4, 4, 32)  │     27,680 │ concatenate_10[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_8     │ (None, 8, 8, 32)  │          0 │ conv2d_44[0][0]   │
│ (UpSampling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_11      │ (None, 8, 8, 64)  │          0 │ up_sampling2d_8[… │
│ (Concatenate)       │                   │            │ max_pooling2d_23… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_45 (Conv2D)  │ (None, 8, 8, 32)  │     18,464 │ concatenate_11[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_9     │ (None, 16, 16,    │          0 │ conv2d_45[0][0]   │
│ (UpSampling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_12      │ (None, 16, 16,    │          0 │ up_sampling2d_9[… │
│ (Concatenate)       │ 64)               │            │ max_pooling2d_22… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 16, 16,    │     18,464 │ concatenate_12[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_10    │ (None, 32, 32,    │          0 │ conv2d_46[0][0]   │
│ (UpSampling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 32, 32,    │      9,248 │ up_sampling2d_10… │
│                     │ 32)               │            │                 

 Total params: 112,033 (437.63 KB)

 Trainable params: 112,033 (437.63 KB)

 Non-trainable params: 0 (0.00 B)